In [1]:
import cv2

In [2]:
#function for assigning class labels

def name_map(x):
    if x == "sachin": #folder name
        return 0
    if x == "obama":
        return 1
    if x == 'messi':
        return 2

In [4]:
#to get the actual name

def get_name(x):
    if x == 0:
        return 'sachin'
    if x == 1:
        return 'obama'
    if x == 2:
        return 'messi'


In [5]:
#detect faces from img- region of ineterst(roi)

def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(r'F:\DL Projects\Face_Recognition\haarcascade_frontalface_default.xml')
    face = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)

    if len(face) == 0: #if no face is detected
        return None, None #this func actually returns 2 values
    #in all these img we have only one face
    #using slicing method extract only face
    x,y,w,h = face[0]
    return gray[x:x+w,y:y+h],face[0]

In [6]:
import os

In [7]:
#preparing train data

def prepare_training_data(data_folder_path):
    dirs = os.listdir(data_folder_path) #to find subfolders
    #print(dirs)
    faces = [] #to store faces
    labels = [] #pass labels
    for dir_name in dirs:
        sub_dir_path = data_folder_path+'/'+dir_name #other slash wont work
        #print(sub_dir_path)
        image_names = os.listdir(sub_dir_path)
        #print(image_names)
        for img_name in image_names:
            img_path = sub_dir_path+'/'+img_name
            #print(img_path)

            #display

            img = cv2.imread(img_path)
            cv2.imshow("Training Image", img)
            cv2.waitKey(10) #after 10ms new pic comes
            
            
            #detect face from every img
            #use the fun
            face, rect = detect_face(img)
            if face is not None:
                faces.append(face)
                labels.append(name_map(dir_name))




    cv2.waitKey(1) #after all pics wait 1 ms then destroy the window
    cv2.destroyAllWindows()
    return faces, labels

        


In [8]:
path = r"F:\DL Projects\Face_Recognition\DB"
faces, labels = prepare_training_data(path)
print(labels)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
len(labels)

29

In [10]:
len(faces)

29

from one sachin img face is not detected. 

In [11]:
#face recognize
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer

< cv2.face.LBPHFaceRecognizer 0000007FBB71BB90>

In [12]:
import numpy as np
recognizer.train(faces, np.array(labels)) #convert labels to array

In [13]:
#draw rect

def draw_rectangle(img,rect):
    (x,y,w,h) = rect
    cv2.rectangle(img, (x,y), (x+w,y+h), (233,231,123), 3)
    

In [14]:
def write_text(img, text, x,y):
    cv2.putText(img, text, (x,y), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,255), 2)

In [15]:
def predict_new(test_img): #to predict a new img
    img = test_img.copy()
    face, rect = detect_face(img)
    label = recognizer.predict(face)
    label_text = get_name(label[0])
    draw_rectangle(img,rect)
    write_text(img,label_text,rect[0],rect[1])
    return img


In [16]:
test_img = cv2.imread(r"F:\DL Projects\Face_Recognition\obama.jpg")
test_img

array([[[ 24,  33,  83],
        [ 15,  27,  81],
        [ 16,  31,  93],
        ...,
        [ 20,  27, 100],
        [ 25,  32, 105],
        [ 40,  44, 127]],

       [[ 28,  32,  83],
        [ 19,  26,  81],
        [ 18,  31,  93],
        ...,
        [ 20,  27, 100],
        [ 25,  32, 105],
        [ 40,  44, 127]],

       [[ 34,  32,  84],
        [ 24,  25,  81],
        [ 22,  29,  92],
        ...,
        [ 20,  27, 100],
        [ 25,  32, 105],
        [ 40,  44, 127]],

       ...,

       [[174, 174, 204],
        [167, 169, 193],
        [180, 183, 198],
        ...,
        [ 19,  15,  26],
        [ 14,   8,  19],
        [ 26,  20,  31]],

       [[133, 133, 169],
        [161, 162, 190],
        [188, 191, 206],
        ...,
        [ 30,  16,  17],
        [ 32,  16,  17],
        [ 33,  17,  18]],

       [[130, 129, 168],
        [156, 158, 188],
        [186, 189, 204],
        ...,
        [ 37,  17,  12],
        [ 37,  17,  12],
        [ 38,  18,  13]]

In [17]:
predicted = predict_new(test_img)
 #2nd value is the confidence, we dont need that
cv2.imshow("Predicted Image",predicted)
cv2.waitKey()
cv2.destroyAllWindows()